In [1]:
import numpy as np
import pandas as pd

def raw_data(path):
  data = pd.read_csv(path)
  return data.values


In [2]:
def parzenWindowd(u, sigma):
    if np.abs(u) <= np.abs(1/2):
        return 1
    else:
        return 0

In [3]:
import math

def gaussian(u, sigma):
   #   if  np.abs(u) <=  np.abs(1/2):
      return (1/(math.sqrt(2*math.pi) * sigma)) * math.exp(-(u**2)/(2*sigma**2))
   #   else:
      #   return 0


In [48]:
def kernel(kernelFunction, dataPoint, x, h, sigma):
    dim = len(dataPoint)
    prod = 1
    for j in range(0, dim):
        prod *= kernelFunction((x[j]-dataPoint[j])/h, sigma)
    return prod


def KDE(data_2d, h,X_2d, sigma, kernel_type='gaussian'):
    N = np.size(X_2d, 0)
    d = np.size(data_2d, 1)  
    probs = []  

    if kernel_type == 'parzen':
        K = parzenWindowd
    elif kernel_type == 'gaussian':
        K = gaussian

    for x in X_2d:
        px = 1/N * 1/(h**d) * np.sum([kernel(K, dataPoint, x, h, sigma) for dataPoint in data_2d])
        probs.append(px) 

    return np.array(probs)

In [46]:
from sklearn.model_selection import train_test_split
data = raw_data('dataset.csv')
X_train, X_test, y_train, y_test = train_test_split(data[:,:-1], data[:,-1], test_size=0.1, random_state=42)

class0Train = []
class1Train = []
class2Train = []

for index, label in enumerate(y_train):
    if label == 0:
        class0Train.append(X_train[index])
    elif label == 1:
        class1Train.append(X_train[index])
    elif label == 2:
        class2Train.append(X_train[index])

class0Train = np.array(class0Train)
class1Train = np.array(class1Train)
class2Train = np.array(class2Train)

class0Py = len(X_train) / len(class0Train)
class1Py = len(X_train) / len(class1Train)
class2Py = len(X_train) / len(class2Train)
print(class2Py)


3.0066815144766146


In [43]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Hs = [0.3, 0.6, 0.9]
sigma = 0.6

In [49]:
from sklearn.metrics import accuracy_score

for h in Hs:
    print("h: ",h)
    class0Prob = KDE(class0Train, h, X_test, sigma, 'gaussian') * class0Py
    class1Prob = KDE(class1Train, h, X_test, sigma, 'gaussian') * class1Py
    class2Prob = KDE(class2Train, h, X_test, sigma, 'gaussian') * class2Py



    y_pred = []
    for i,val in enumerate(class0Prob):
        maxi = val
        label = 0
        if class1Prob[i] > maxi:
            maxi = class1Prob[i]
            label = 1
        if class2Prob[i]> maxi:
            maxi = class2Prob[i]
            label = 2
        y_pred.append(label)
    y_pred = np.array(y_pred)

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: ",accuracy * 100)

h:  0.3
Accuracy:  88.66666666666667
h:  0.6
Accuracy:  90.66666666666666
h:  0.9
Accuracy:  91.33333333333333
